给定一个RPA流程需求描述，先让LLM重新描述一个需求和执行步骤，然后生成伪代码，再根据API文档和伪代码，生成实际的代码。

In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
print(openai.api_type, openai.api_base, openai.api_version)

llm = ChatOpenAI(engine='test0406', model_name="gpt-3.5-turbo", temperature=0.3, max_tokens=4096)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


azure https://test0406.openai.azure.com/ 2023-03-15-preview


In [3]:
task_description = '''通过RPA机器人，进入网银系统获取前一个工作日的资金流水数据，
按数据类型写入到模板表对应的银行列中，并下载前一工作日电子回单，生成文件清单并打印，
再发送资金日报及电子回单清单给对应的业务人员。
'''

task_steps = '''操作步骤及说明
1.	业务人员配置好银行账号数据表，和邮箱账号数据；
2.	读取账号配置表，根据账号信息自动登录网银系统；
3.	获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏；
4.	下载前一天电子回单、网银流水，按照固定规则命名（命名规则：公司_收/付_账号_日期，例：X公司_收_账号_20211222），并在日志记录表中写入记录；
5.	将日报excel及回、流水文件夹进行压缩；
6.	邮件发送运行异常提醒，将运行的数据压缩文件发送到指定邮箱；
7.	业务人员接收邮件文件，进行确认操作。
'''

In [4]:
template_task = '''你是个非常优秀的RPA程序员，你的任务是根据RPA流程的需求描述和步骤描述，生成一个伪代码表示的RPA流程。
请一步步思考，确保正确理解实现该流程的所有必要条件，然后更新步骤描述，确保新的步骤描述包含了所有的上下文信息，以方便后面的代码。

流程描述: {description}

任务步骤：{steps}

重新描述实现该RPA流程的具体步骤，包括每个步骤的输入输出和实现功能：'''

prompt_task = PromptTemplate(input_variables=['description', 'steps'], template=template_task)
task_chain = LLMChain(llm=llm, prompt=prompt_task, output_key='revised')

In [5]:
template_draft = '''你是个非常优秀的RPA程序员，请根据RPA流程描述和实现步骤描述，先生成一个用伪代码表示的RPA流程草稿。

流程描述: {description}

详细步骤：{revised}

生成的伪代码RPA流程是：'''

prompt_draft = PromptTemplate(input_variables=['description', 'revised'], template=template_draft)
draft_chain = LLMChain(llm=llm, prompt=prompt_draft, output_key='draft')

In [6]:
template_code = '''你是个非常优秀的RPA程序员，请根据RPA流程描述和流程草稿，实现一个伪代码表示的RPA流程。

流程描述: {description}
流程草稿: {draft}

生成的RPA流程伪代码：'''

prompt_code = PromptTemplate(input_variables=['description', 'draft'], template=template_code)

code_chain = LLMChain(llm=llm, prompt=prompt_code, output_key='code')

In [7]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[task_chain, draft_chain, code_chain],
    input_variables=['description', 'steps'],
    # Here we return multiple variables
    output_variables=["revised", "draft", "code"],
    verbose=True)

In [8]:
result = overall_chain({'description': task_description, 'steps': task_steps})



> Entering new SequentialChain chain...

> Finished chain.


In [9]:
print(result['revised'])

1. 业务人员配置好银行账号数据表和邮箱账号数据；
   - 输入：银行账号数据表和邮箱账号数据；
   - 输出：无；
   - 功能：配置银行账号数据表和邮箱账号数据。

2. 读取账号配置表，根据账号信息自动登录网银系统；
   - 输入：银行账号数据表；
   - 输出：无；
   - 功能：根据银行账号数据表中的账号信息，自动登录网银系统。

3. 获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏；
   - 输入：前一天历史余额数据；
   - 输出：资金日报表模板对应银行账户余额栏；
   - 功能：获取前一天历史余额数据，并将其写入资金日报表模板对应银行账户余额栏。

4. 下载前一天电子回单、网银流水，按照固定规则命名（命名规则：公司_收/付_账号_日期，例：X公司_收_账号_20211222），并在日志记录表中写入记录；
   - 输入：前一天电子回单和网银流水；
   - 输出：按照固定规则命名的电子回单和网银流水，日志记录表；
   - 功能：下载前一天电子回单和网银流水，并按照固定规则命名，同时在日志记录表中写入记录。

5. 将日报excel及回、流水文件夹进行压缩；
   - 输入：资金日报表模板对应银行账户余额栏、按照固定规则命名的电子回单和网银流水；
   - 输出：压缩后的日报excel及回、流水文件夹；
   - 功能：将资金日报表模板对应银行账户余额栏、按照固定规则命名的电子回单和网银流水进行压缩。

6. 邮件发送运行异常提醒，将运行的数据压缩文件发送到指定邮箱；
   - 输入：压缩后的日报excel及回、流水文件夹、邮箱账号数据；
   - 输出：无；
   - 功能：将压缩后的日报excel及回、流水文件夹发送到指定邮箱，并发送运行异常提醒。

7. 业务人员接收邮件文件，进行确认操作。
   - 输入：邮件文件；
   - 输出：无；
   - 功能：业务人员接收邮件文件，并进行确认操作。


In [10]:
print(result['draft'])

1. 读取银行账号数据表和邮箱账号数据；
2. 自动登录网银系统；
3. 获取前一天历史余额数据，并将其写入资金日报表模板对应银行账户余额栏；
4. 下载前一天电子回单和网银流水，并按照固定规则命名，同时在日志记录表中写入记录；
5. 将资金日报表模板对应银行账户余额栏、按照固定规则命名的电子回单和网银流水进行压缩；
6. 将压缩后的日报excel及回、流水文件夹发送到指定邮箱，并发送运行异常提醒；
7. 业务人员接收邮件文件，并进行确认操作。


In [11]:
print(result['code'])

1. Read bank account data table and email account data
2. Login to online banking system automatically
3. Get historical balance data from the previous day and write it to the corresponding bank account balance column in the financial report template
4. Download electronic receipts and online banking transactions from the previous day, name them according to a fixed rule, and write a record in the log
5. Compress the bank account balance column in the financial report template, the named electronic receipts and online banking transactions according to a fixed rule
6. Send the compressed financial report excel and the folder containing the named electronic receipts and online banking transactions to the designated email address, and send a notification if there is an exception during the process
7. The business personnel receive the email files and perform confirmation operations.


In [12]:
# with open('patents/requirements-test4.txt', 'w') as f:
#     f.write('requirements: ' + result['requirements'])
#     f.write('\n\nfeature: ' + result['feature'])
#     f.write('\n\nsolution: ' + result['solution'])
#     f.write('\n\nselection: ' + result['selection'])

In [13]:
steps = result['code'].split('\n')
steps[:3]

['1. Read bank account data table and email account data',
 '2. Login to online banking system automatically',
 '3. Get historical balance data from the previous day and write it to the corresponding bank account balance column in the financial report template']

In [14]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI


embeddings = OpenAIEmbeddings(model='text-embedding-ada-002',
                              deployment='emb0614',
                              openai_api_base=os.getenv("OPENAI_API_BASE"),
                              openai_api_type=os.getenv("OPENAI_API_TYPE"),
                              chunk_size=1)

docsearch = Chroma(persist_directory='apidocs', embedding_function=embeddings)

In [17]:
template_generate = """你是一个专业的RPA开发人员，请根据RPA库函数描述，选择合适的函数来生成一个任务步骤的代码。
如果实现这个任务步骤所需要的函数不存在于上下文中，则可以使用通用第三方函数库。如果存在，则必须使用已有的RPA库函数。

相关函数: {context}

当前步骤：{question}

生成的代码:"""

prompt_generate = PromptTemplate(
    template=template_generate, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": prompt_generate}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)

In [18]:
result = qa.run(steps[0])

"// Read bank account data table\nconst bankData = readDataTable('bankData', { headers: true });\n\n// Read email account data\nconst emailData = readDataTable('emailData', { headers: true });\n\n// Retrieve emails from Exchange email account\nconst emails = receiveExchangeEmail(emailData.serverAddress, emailData.adDomain, emailData.username, emailData.password, emailData.emailAddress, emailData.folder, emailData.senderFilter, emailData.subjectFilter, emailData.contentFilter, true, true, emailData.attachmentSavePath, true, 10);"

In [20]:
print(result)

// Read bank account data table
const bankData = readDataTable('bankData', { headers: true });

// Read email account data
const emailData = readDataTable('emailData', { headers: true });

// Retrieve emails from Exchange email account
const emails = receiveExchangeEmail(emailData.serverAddress, emailData.adDomain, emailData.username, emailData.password, emailData.emailAddress, emailData.folder, emailData.senderFilter, emailData.subjectFilter, emailData.contentFilter, true, true, emailData.attachmentSavePath, true, 10);


In [21]:
for i in range(len(steps)):
    result = qa.run(steps[i])
    print(result)

// Read bank account data table
const bankAccounts = readDataTable('bankAccountsTable');

// Read email account data
const emailAccount = readDataTable('emailAccountTable')[0];

// Retrieve emails from inbox folder
const emails = receiveExchangeEmail(emailAccount.serverAddress, emailAccount.adDomain, emailAccount.username, emailAccount.password, emailAccount.emailAddress, 'inbox', null, 'Bank statement', null, true, true, null, true, null);

// Loop through retrieved emails
for (const email of emails) {
  // Check if email contains bank account number
  for (const account of bankAccounts) {
    if (email.body.includes(account.accountNumber)) {
      // Do something with the email
      break;
    }
  }
}
autoSFTPLogin('bankingserver.com', '22', 'username', 'password')
calculateDateDifference('previous_day', 1, '%Y-%m-%d', '%Y-%m-%d')
appendDataToCSV('historical_balance_data.csv', 'bank_account_balance_column', 'financial_report_template.csv')
setDataRecordState('historical_balance_data